In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.plotly as py
#from chart_studio import plotly
import plotly.graph_objs as go
#from plotly.plotly import plot_mpl
#importing plotly and cufflinks in offline mode
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from pylab import rcParams
import cufflinks as cf
init_notebook_mode(connected = True)
import plotly.offline
cf.go_offline()
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

#from pandas.tools.plotting import autocorrelation_plot
from pandas.plotting import autocorrelation_plot 
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import itertools
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pip install iplot

In [3]:
pip install cufflinks

Processing /Users/gaby/Library/Caches/pip/wheels/e1/27/13/3fe67fa7ea7be444b831d117220b3b586b872c9acd4df480d0/cufflinks-0.17.3-py3-none-any.whl
  Using cached colorlover-0.3.0-py3-none-any.whl (8.9 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import plotly.offline as pyoff
import plotly.graph_objs as go

In [ ]:
pip install chart_studio

In [6]:
df = pd.read_csv("zillow_data.csv")
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1/31/2000,...,7/31/2020,8/31/2020,9/30/2020,10/31/2020,11/30/2020,12/31/2020,1/31/2021,2/28/2021,3/31/2021,4/30/2021
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,327302.0,...,1044070,1059557,1067331,1068534.0,1069807,1081347,1081855,1092196.0,1104487.0,1121360.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,314318.0,...,498824,500922,503725,506441.0,508991,511451,513654,515983.0,517482.0,519569.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,380271.0,...,1131516,1133589,1144334,1144987.0,1140435,1130923,1121519,1120821.0,1118048.0,1120428.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,223127.0,...,343268,344710,346294,348806.0,351884,356253,360496,365462.0,370763.0,376914.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,407203.0,...,637989,639520,642386,645129.0,648104,650989,653530,656818.0,659416.0,662782.0


In [7]:
# Remove null values
df = df.dropna()

In [8]:
df.rename(columns={"RegionName":"ZipCode"}, inplace=True)
df["ZipCode"]=df["ZipCode"].map(lambda x: "{:.0f}".format(x))
df["RegionID"]=df["RegionID"].map(lambda x: "{:.0f}".format(x))
df.head()

,RegionID,SizeRank,ZipCode,RegionType,StateName,State,City,Metro,CountyName,1/31/2000,...,7/31/2020,8/31/2020,9/30/2020,10/31/2020,11/30/2020,12/31/2020,1/31/2021,2/28/2021,3/31/2021,4/30/2021
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,327302.0,...,1044070,1059557,1067331,1068534.0,1069807,1081347,1081855,1092196.0,1104487.0,1121360.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,314318.0,...,498824,500922,503725,506441.0,508991,511451,513654,515983.0,517482.0,519569.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,380271.0,...,1131516,1133589,1144334,1144987.0,1140435,1130923,1121519,1120821.0,1118048.0,1120428.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,223127.0,...,343268,344710,346294,348806.0,351884,356253,360496,365462.0,370763.0,376914.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,407203.0,...,637989,639520,642386,645129.0,648104,650989,653530,656818.0,659416.0,662782.0


In [9]:
df_usa = df.loc[:,'4/30/2000':'4/30/2021']
df_usa = df_usa.transpose()
df_usa.head()

,0,1,2,3,4,5,6,7,8,9,...,30802,30804,30814,30816,30821,30822,30827,30828,30830,30831
4/30/2000,348716.0,326931.0,394224.0,227925.0,421469.0,106485.0,426013.0,105782.0,89811.0,195800.0,...,276944.0,359284.0,99940.0,218641.0,326623.0,450151.0,1837726.0,314142.0,113091.0,221538.0
5/31/2000,356557.0,331085.0,400534.0,227131.0,426227.0,106531.0,429756.0,105823.0,89806.0,198606.0,...,279522.0,360945.0,101149.0,219641.0,329608.0,450598.0,1871906.0,299144.0,111687.0,222853.0
6/30/2000,363894.0,335104.0,408671.0,226349.0,430855.0,106697.0,431719.0,105954.0,89676.0,201307.0,...,279184.0,361550.0,102499.0,224039.0,333533.0,452551.0,1902100.0,284628.0,111674.0,223848.0
7/31/2000,370953.0,339022.0,416193.0,225192.0,435420.0,106801.0,437733.0,105960.0,89558.0,203630.0,...,281396.0,364551.0,103853.0,226287.0,335948.0,459797.0,1910204.0,270764.0,110666.0,224326.0
8/31/2000,376962.0,343108.0,423100.0,225702.0,439915.0,107065.0,440354.0,106187.0,89446.0,206387.0,...,284001.0,369502.0,105818.0,230468.0,345577.0,464686.0,1930185.0,264862.0,108143.0,224817.0


In [10]:
usa_price = df_usa.mean(axis=1)
usa_price = pd.DataFrame(usa_price)
usa_price = usa_price.reset_index()
usa_price = usa_price.rename(columns={'index':'Time', 0:'Average Price'})
usa_price['Time'] = pd.to_datetime(usa_price['Time'])
usa_price.set_index('Time', inplace=True)
usa_price.head()

,Average Price
Time,
2000-04-30,152603.319032
2000-05-31,153671.983864
2000-06-30,154744.463694
2000-07-31,155805.848771
2000-08-31,156941.502470


In [11]:
usa_price.iplot(title="The USA Single Family Home Prices 2001-2021",
                    xTitle="Year",
                    yTitle="Sales Price",
                    shape=(12,1)
                    )

In [12]:
df_state = df.copy()

In [13]:
df_state['row_mean'] = df_usa.mean(axis=0) 

In [14]:
df_state.head()

,RegionID,SizeRank,ZipCode,RegionType,StateName,State,City,Metro,CountyName,1/31/2000,...,8/31/2020,9/30/2020,10/31/2020,11/30/2020,12/31/2020,1/31/2021,2/28/2021,3/31/2021,4/30/2021,row_mean
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,327302.0,...,1059557,1067331,1068534.0,1069807,1081347,1081855,1092196.0,1104487.0,1121360.0,816090.762846
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,314318.0,...,500922,503725,506441.0,508991,511451,513654,515983.0,517482.0,519569.0,452310.537549
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,380271.0,...,1133589,1144334,1144987.0,1140435,1130923,1121519,1120821.0,1118048.0,1120428.0,858449.786561
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,223127.0,...,344710,346294,348806.0,351884,356253,360496,365462.0,370763.0,376914.0,283558.241107
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,407203.0,...,639520,642386,645129.0,648104,650989,653530,656818.0,659416.0,662782.0,577159.992095


In [15]:
df_state_price = df_state[['State', 'row_mean']]
df_state_avg_price = df_state_price.groupby(['State'], as_index=False).mean()
df_state_avg_price = df_state_avg_price.rename(columns={'State': 'Code', 'row_mean': 'State Housing Price'})
df_state_avg_price['State Housing Price'] = df_state_avg_price['State Housing Price'].round(2)
df_state_avg_price.head()

,Code,State Housing Price
0,AK,247182.18
1,AL,128518.45
2,AR,96411.08
3,AZ,211610.51
4,CA,522087.13


In [16]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [17]:
us_state_code = pd.DataFrame.from_dict(us_state_abbrev, orient='index')
us_state_code = us_state_code.reset_index()
us_state_code = us_state_code.rename(columns={'index': 'State', 0: 'Code'})
df_state_avg_price = df_state_avg_price.merge(us_state_code, on='Code', how='inner')
df_state_avg_price['Description'] = df_state_avg_price['State'].map(str) + '-' + df_state_avg_price['State Housing Price'].map(str)
df_state_avg_price.head()

,Code,State Housing Price,State,Description
0,AK,247182.18,Alaska,Alaska-247182.18
1,AL,128518.45,Alabama,Alabama-128518.45
2,AR,96411.08,Arkansas,Arkansas-96411.08
3,AZ,211610.51,Arizona,Arizona-211610.51
4,CA,522087.13,California,California-522087.13


In [18]:
data = dict(type = 'choropleth', 
            colorscale = 'Picnic', 
            locations = df_state_avg_price['Code'], 
            z = df_state_avg_price['State Housing Price'], 
            locationmode = 'USA-states', 
            text = df_state_avg_price['Description'], 
            marker = dict(line = dict(color = 'rgb(255, 255,255)', width = 2)),
            colorbar = {'title':"Housing Price"}
           )

layout = dict(title = 'The USA Housing Price',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )

choromap = go.Figure(data = [data], layout=layout)

iplot(choromap)

In [17]:
df_2018 = df.loc[:, '12/31/2019':'12/31/2020']
df_state = pd.DataFrame(df['State'])
df_recent = pd.concat([df_state, df_2018], axis=1)
df_recent['row_mean'] = df_recent.mean(axis=1) 
df_state_recent_price = df_recent[['State', 'row_mean']]
df_state_recent_avg_price = df_state_recent_price.groupby(['State'], as_index=False).mean()
df_state_recent_avg_price = df_state_recent_avg_price.rename(columns={'State': 'Code', 'row_mean': '2020 State Housing Price'})
df_state_recent_avg_price['2020 State Housing Price'] = df_state_recent_avg_price['2020 State Housing Price'].round(2)
df_state_recent_avg_price.head()

,Code,2020 State Housing Price
0,AK,300314.31
1,AL,156181.58
2,AR,120551.51
3,AZ,290685.23
4,CA,724163.66


In [18]:
df_state_recent_avg_price = df_state_recent_avg_price.merge(us_state_code, on='Code', how='inner')
df_state_recent_avg_price['Description'] = df_state_recent_avg_price['State'].map(str) + '-' + df_state_recent_avg_price['2020 State Housing Price'].map(str)
df_state_recent_avg_price.head()

,Code,2020 State Housing Price,State,Description
0,AK,300314.31,Alaska,Alaska-300314.31
1,AL,156181.58,Alabama,Alabama-156181.58
2,AR,120551.51,Arkansas,Arkansas-120551.51
3,AZ,290685.23,Arizona,Arizona-290685.23
4,CA,724163.66,California,California-724163.66


In [19]:
data = dict(type = 'choropleth', 
            colorscale = 'Picnic', 
            locations = df_state_recent_avg_price['Code'], 
            z = df_state_recent_avg_price['2020 State Housing Price'], 
            locationmode = 'USA-states', 
            text = df_state_recent_avg_price['Description'], 
            marker = dict(line = dict(color = 'rgb(255, 255,255)', width = 2)),
            colorbar = {'title':"2020 Housing Price"}
           )

layout = dict(title = 'The USA 2020 Housing Price',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )

choromap = go.Figure(data = [data], layout=layout)

iplot(choromap)

In [ ]:
df_price = df.loc[:, '4/30/2000':'4/30/2021']
df_state = pd.DataFrame(df['State'])
df_state_price = pd.concat([df_state, df_price], axis=1)
df_state_price = df_state_price.rename(columns={'State':'Code'})
df_state_price.head()

In [ ]:
df_state_price = df_state_price.merge(us_state_code, on='Code', how='inner')
df_state_price.head()

In [ ]:
df_state_price_trend = df_state_price.groupby(['State']).mean()
df_state_price_trend = df_state_price_trend.transpose()
df_state_price_trend.head()

In [ ]:
df_state_price_trend.iplot(title="The States' Single Family Home Prices 2001-2021",
                    xTitle="Year",
                    yTitle="Sales Price",
                    shape=(12,1)
                    )

In [ ]:
#def main():
    #print('Please input the state name you want to show')
    #state_name = input()
state_name = 'Georgia'
one_state_price_trend = pd.DataFrame(df_state_price_trend[state_name])
state_usa_price_trend = pd.concat([one_state_price_trend, usa_price], axis=1) # usa_price is the above one
state_usa_price_trend = state_usa_price_trend.rename(columns={'Average Price': 'USA Average Price'})

state_usa_price_trend.iplot(title=f"The {state_name} Single Family Home Prices 2001-2021",
                    xTitle="Year",
                    yTitle="Sales Price",
                    shape=(12,1)
                    )
#if __name__ == "__main__":
    #main()

In [ ]:
#def main():
#print('Please input the city name you want to show')
    #city_name = input()
city_name = 'Atlanta'
df_price = df.loc[:, '4/30/2001':'4/30/2021']
df_city = pd.DataFrame(df['City'])
df_city_price = pd.concat([df_city, df_price], axis=1)
df_city_avg_price = df_city_price.groupby(['City']).mean()
df_city_avg_price = df_city_avg_price.transpose()
one_city_price_trend = pd.DataFrame(df_city_avg_price[city_name])
city_usa_price_trend = pd.concat([one_city_price_trend, usa_price], axis=1) # usa_price is the above one
city_usa_price_trend = city_usa_price_trend.rename(columns={'Average Price': 'USA Average Price'})
    
city_usa_price_trend.iplot(title=f"The {city_name} Single Family Home Prices 2001-2021",
                    xTitle="Year",
                    yTitle="Sales Price",
                    shape=(12,1)
                    )
#if __name__ == "__main__":
    #main()

In [ ]:
usa_price_trend = usa_price.copy()
usa_price_trend = usa_price_trend.round(2)
usa_price_trend.head()

In [ ]:
rolmean = usa_price_trend.rolling(12).mean()
rolstd = usa_price_trend.rolling(12).std()

#Plot rolling statistics:
fig = plt.figure(figsize=(12, 6))
orig = plt.plot(usa_price_trend, color='blue',label='Original')
mean = plt.plot(rolmean, color='red', label='Rolling Mean')
std = plt.plot(rolstd, color='black', label = 'Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

In [ ]:
usa_decomposition = sm.tsa.seasonal_decompose(usa_price_trend, model='additive')
rcParams['figure.figsize'] = 18, 8
fig = usa_decomposition.plot()

In [ ]:
p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

warnings.filterwarnings("ignore") 

grid_results= []
for param in pdq:
    for seasonal_param in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(usa_price_trend,
                                            order=param,
                                            seasonal_order=seasonal_param,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            model = mod.fit()
            grid_results.append([param, seasonal_param, model.aic])
        except:
            print('error')
            continue

grid_results

In [ ]:
model = sm.tsa.statespace.SARIMAX(usa_price_trend,
                                 order=(1,1,1), 
                                 seasonal_order=(1,1,0,12),   
                                 enforce_stationarity=False,
                                 enforce_invertibility=False)
result = model.fit()
print(result.summary())

In [ ]:
result.plot_diagnostics(figsize=(12, 12))
plt.show()

In [ ]:
pred_dynamic = result.get_prediction(start=pd.to_datetime('4-30-2000'), dynamic=True, full_results=True)
pred_dynamic_conf_int = pred_dynamic.conf_int()

axes = usa_price_trend['2000':].plot(label='Observed', figsize=(18, 10))
pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=axes)
axes.fill_between(pred_dynamic_conf_int.index,
                pred_dynamic_conf_int.iloc[:, 0],
                pred_dynamic_conf_int.iloc[:, 1], 
                color='r', 
                alpha=.3)
axes.fill_betweenx(axes.get_ylim(), 
                  pd.to_datetime('2014'), 
                  usa_price_trend.index[-1],
                  alpha=.1, zorder=-1)

axes.set_xlabel('Time (years)')
axes.set_ylabel('Average Housing Price')
plt.legend()

In [ ]:
usa_forecast = result.get_forecast(steps= 60)
usa_forecast_conf_int = usa_forecast.conf_int()

axes = usa_price_trend.plot(label='Observed', figsize=(18, 10))
usa_forecast.predicted_mean.plot(ax=axes, label='Forecast')
axes.fill_between(usa_forecast_conf_int.index,
                 usa_forecast_conf_int.iloc[:, 0],
                 usa_forecast_conf_int.iloc[:, 1], color='b', alpha=.4)
axes.set_xlabel('Time')
axes.set_ylabel('USA Housing Price')
plt.legend()

In [ ]:
df_state_price_trend.head()

In [ ]:
df_state_price_trend = df_state_price_trend.round(2)
df_state_price_trend.head()

In [ ]:
ca_price_trend = pd.DataFrame(df_state_price_trend['Georgia'])
ca_price_trend = ca_price_trend.reset_index()
ca_price_trend['index'] = pd.to_datetime(ca_price_trend['index'])
ca_price_trend = ca_price_trend.set_index('index')
ca_price_trend.head()

In [ ]:
rolmean = ca_price_trend.rolling(12).mean()
rolstd = ca_price_trend.rolling(12).std()

#Plot rolling statistics:
fig = plt.figure(figsize=(12, 6))
orig = plt.plot(ca_price_trend, color='blue',label='Original')
mean = plt.plot(rolmean, color='red', label='Rolling Mean')
std = plt.plot(rolstd, color='black', label = 'Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

In [ ]:
ca_decomposition = sm.tsa.seasonal_decompose(ca_price_trend, model='additive')
rcParams['figure.figsize'] = 18, 8
fig = ca_decomposition.plot()

In [ ]:
p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

warnings.filterwarnings("ignore") 

grid_results= []
for param in pdq:
    for seasonal_param in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(ca_price_trend,
                                            order=param,
                                            seasonal_order=seasonal_param,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            model = mod.fit()
            grid_results.append([param, seasonal_param, model.aic])
        except:
            print('error')
            continue

grid_results

In [ ]:
model = sm.tsa.statespace.SARIMAX(ca_price_trend,
                                 order=(1,1,1), 
                                 seasonal_order=(0,1,1,12),   
                                 enforce_stationarity=False,
                                 enforce_invertibility=False)
result = model.fit()
print(result.summary())

In [ ]:
result.plot_diagnostics(figsize=(12, 12))
plt.show()

In [ ]:
pred_dynamic = result.get_prediction(start=pd.to_datetime('4/30/2018'), dynamic=True, full_results=True)
pred_dynamic_conf_int = pred_dynamic.conf_int()

axes = ca_price_trend['4/30/2000':].plot(label='Observed', figsize=(18, 10))
pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=axes)
axes.fill_between(pred_dynamic_conf_int.index,
                pred_dynamic_conf_int.iloc[:, 0],
                pred_dynamic_conf_int.iloc[:, 1], 
                color='r', 
                alpha=.3)
axes.fill_betweenx(axes.get_ylim(), 
                  pd.to_datetime('04/30/2021'), 
                  ca_price_trend.index[-1],
                  alpha=.1, zorder=-1)

axes.set_xlabel('Time (years)')
axes.set_ylabel('California Average Housing Price')
plt.legend()